In [1]:
import uproot
import awkward as ak

import numpy as np

np.random.seed(0)

In [2]:
sequences = uproot.open("data/output_1000_seq3.root:L1BMTFStubSequences").arrays()
sequences

<Array [{nL1BMTFStub: 1, ...}, ..., {...}] type='158919 * {nL1BMTFStub: int...'>

In [3]:
def orbit_aggregator(data: ak.Array):
    assert "orbitNumber" in data.fields
    
    data_sorted = data[ak.argsort(data["orbitNumber"])]
    orbit_reps = ak.run_lengths(data_sorted["orbitNumber"])

    """
    unflatten data
    """
    data_uf = ak.unflatten(
        data_sorted, 
        orbit_reps
    )
    
    """
    group by orbit (gpo)
    """
    zip_dict = {key: data_uf[key] for key in data_uf.fields if key != "orbitNumber"}
    zip_dict["orbitNumber"] = data_uf["orbitNumber"][:,0]
    data_gpo = ak.zip(zip_dict, depth_limit=1)

    return data_gpo.to_list()

sequences_gpo = orbit_aggregator(sequences)

In [4]:
def shuffle_orbit_sequences(orbit_dict):
    shuffle_pattern = np.random.permutation(len(orbit_dict["bunchCrossing"]))
    shuffled_bxs = ak.Array(orbit_dict["bunchCrossing"])[shuffle_pattern]

    while np.any(np.diff(shuffled_bxs) == 1):
        shuffle_pattern = np.random.permutation(len(orbit_dict["bunchCrossing"]))
        shuffled_bxs = ak.Array(orbit_dict["bunchCrossing"])[shuffle_pattern]
        
    shuffled_orbit_dict = {}

    for key in orbit_dict:
        if key == "orbitNumber":
            continue
        
        shuffled_orbit_dict[key] = ak.Array(orbit_dict[key])[shuffle_pattern].to_list()

    shuffled_orbit_dict["orbitNumber"] = orbit_dict["orbitNumber"]
    return shuffled_orbit_dict

shuffled_sequences_gpo = list(map(shuffle_orbit_sequences, sequences_gpo))

In [5]:
def extract_sequences(orbit_dict):
    nbx = len(orbit_dict["bunchCrossing"])
    nseq = nbx // 3
    mask = np.arange(nbx).reshape((nseq, 3))

    sequence_dict = {}

    for key, values in orbit_dict.items():
        if key == "orbitNumber":
            continue

        values_ak = ak.Array(values)
        sequence_dict[key] = values_ak[mask].to_list()

    orbit_number = orbit_dict["orbitNumber"] * np.ones((nseq, 3), dtype=np.int32)
    sequence_dict["orbitNumber"] = orbit_number.tolist()
    return sequence_dict

shuffled_sequences = list(map(extract_sequences, shuffled_sequences_gpo))

In [6]:
shuffled_sequences_conc = ak.concatenate([ak.Array(d) for d in shuffled_sequences]).to_list()

In [7]:
# def bx_check(sequence_dict):
#     bx_diff = np.abs(np.diff(sequence_dict["bunchCrossing"]))

#     if np.all(bx_diff >= 24):
#         return sequence_dict
    

# shuffled_sequences_filtered = list(map(bx_check, shuffled_sequences_conc))
# shuffled_sequences_filtered = list(filter(lambda x: x is not None, shuffled_sequences_filtered))

In [9]:
def bx_check(sequence_dict):
    bx_diff = np.abs(np.diff(sequence_dict["bunchCrossing"]))

    return np.all(bx_diff >= 24)
    
shuffled_sequences_filtered = list(filter(bx_check, shuffled_sequences_conc))

In [12]:
for index, sequence_dict in enumerate(shuffled_sequences_filtered):
    sequence_dict["sequenceIndex"] = 3 * [index]

In [13]:
shuffled_sequences_filtered = ak.concatenate([ak.Array(d) for d in shuffled_sequences_filtered])

In [19]:
shuffled_sequences_filtered[3]

<Record {nL1BMTFStub: 2, ...} type='{nL1BMTFStub: int64, L1BMTFStub_hwQual:...'>

In [20]:
L1BMTFStub = ak.zip({name[11:]: array for name, array in zip(ak.fields(shuffled_sequences_filtered), ak.unzip(shuffled_sequences_filtered)) if name.startswith("L1BMTFStub_")})

file_out = uproot.recreate(f"data/output_1000_seq3s.root")
file_out["L1BMTFStubSequences"] = {
        "L1BMTFStub": L1BMTFStub, 
        "orbitNumber": shuffled_sequences_filtered["orbitNumber"], 
        "bunchCrossing": shuffled_sequences_filtered["bunchCrossing"], 
        "sequenceIndex": shuffled_sequences_filtered["sequenceIndex"]
    }